# Construindo uma MLP com PyTorch: Classificação do Conjunto de Dados Iris


Nesta parte final do notebook, vamos aplicar o que aprendemos para construir uma rede neural MLP (Perceptron Multicamadas) 
usando PyTorch e treiná-la no conjunto de dados Iris para classificar as espécies de iris com base em suas características.



## Índice

1. [Preprocessamento dos Dados](#preprocessamento-dos-dados)
2. [Construindo uma Rede Neural MLP com PyTorch](#construindo-uma-rede-neural-mlp-com-pytorch)
   - [Definindo a Rede Neural](#definindo-a-rede-neural)
   - [Função de Perda e Otimizador](#funcao-de-perda-e-otimizador)
   - [Treinamento da Rede](#treinamento-da-rede)
   - [Avaliação do Modelo](#avaliacao-do-modelo)
3. [Conclusão e Próximos Passos](#conclusao-e-proximos-passos)



## Preprocessamento dos Dados

Antes de alimentar o conjunto de dados Iris para nossa rede neural, precisamos preprocessá-lo. O preprocessamento de dados 
é um passo crucial na construção de modelos de aprendizado de máquina e inclui tarefas como a normalização dos dados 
(ou seja, colocar todos os recursos na mesma escala), a divisão dos dados em conjuntos de treinamento e teste, e a 
conversão dos rótulos em um formato que pode ser facilmente usado para treinar a rede.


In [1]:
#!pip install torch torchvision torchaudio

In [2]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import torch

# carregar iris dataset
from sklearn.datasets import load_iris
iris = load_iris()

# Carregar dados
X = iris.data
y = iris.target



In [7]:
X[:5], y[:5]

(array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2]]),
 array([0, 0, 0, 0, 0]))

In [8]:
# Normalizar os recursos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [9]:
X_train[:5], y_train[:5]

(array([[-1.50652052,  1.24920112, -1.56757623, -1.3154443 ],
        [-0.17367395,  3.09077525, -1.2833891 , -1.05217993],
        [ 1.03800476,  0.09821729,  0.36489628,  0.26414192],
        [-1.26418478,  0.78880759, -1.22655167, -1.3154443 ],
        [-1.74885626,  0.32841405, -1.39706395, -1.3154443 ]]),
 array([0, 0, 1, 0, 0]))

In [30]:


# Converter arrays numpy em tensores PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long) # Os rótulos são de tipo long
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Criar DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)


## Construindo uma Rede Neural MLP com PyTorch

### Definindo a Rede Neural
Agora que nossos dados estão prontos, podemos definir nossa rede neural. Vamos construir uma rede MLP simples com uma 
camada oculta. PyTorch oferece a classe `nn.Module`, que é a classe base para todos os modelos de rede neural.


In [60]:

import torch.nn as nn
import torch.nn.functional as F

class SimpleMLP(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(SimpleMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) # input_
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Número de recursos de entrada (4 características), número de classes (3 tipos de iris)
input_size = 4
hidden_size = 10
num_classes = 3

# Instanciar o modelo
model = SimpleMLP(input_size, hidden_size, num_classes)
print(model)


SimpleMLP(
  (fc1): Linear(in_features=4, out_features=10, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=10, out_features=3, bias=True)
)


In [65]:
model(X_train_tensor[0])

tensor([0.1279, 0.1254, 0.2883], grad_fn=<ViewBackward0>)


### Função de Perda e Otimizador

Depois de definir a rede neural, precisamos definir uma função de perda e escolher um método de otimização. A função de perda, 
também conhecida como critério, mede o desempenho do modelo em termos de quão bem ele prevê os rótulos verdadeiros. O otimizador 
ajusta os parâmetros do modelo para minimizar a função de perda.


In [58]:

# Definir a função de perda e o otimizador
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01) # lr = learning rate (taxa de aprendizado)



### Treinamento da Rede

Com tudo pronto, agora podemos treinar nossa rede. O treinamento de uma rede neural com PyTorch envolve passar pelos dados 
em épocas, onde em cada época, passamos por cada lote de dados do conjunto de treinamento. Para cada lote, executamos o 
processo de forward pass (calculando a saída do modelo e a perda), backpropagation (calculando os gradientes dos parâmetros) 
e otimização (atualizando os parâmetros com o otimizador).


In [67]:

# Definir o número de épocas
num_epochs = 50

# Armazenar as perdas a cada época
loss_values = []

# Treinamento da rede
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):  
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 30 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')
    
    loss_values.append(loss.item())



### Avaliação do Modelo

Depois de treinar o modelo, queremos avaliar seu desempenho no conjunto de dados de teste. Isso é feito passando as 
características de teste através do modelo e comparando as saídas previstas com os rótulos verdadeiros.


In [68]:

# Avaliar o modelo
model.eval()  # Modo de avaliação

with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Acurácia do modelo no conjunto de treinamento: {100 * correct / total}%')

# Testar o modelo com o conjunto de teste
with torch.no_grad():
    correct = 0
    total = 0
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs.data, 1)
    total = y_test_tensor.size(0)
    correct = (predicted == y_test_tensor).sum().item()

    print(f'Acurácia do modelo no conjunto de teste: {100 * correct / total}%')


Acurácia do modelo no conjunto de treinamento: 27.5%
Acurácia do modelo no conjunto de teste: 36.666666666666664%


In [71]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.4615, -0.3760,  0.3352,  0.1857],
         [ 0.2408, -0.1824,  0.3896, -0.3109],
         [ 0.4700, -0.0204,  0.0446, -0.3956],
         [-0.3670, -0.0942,  0.2002,  0.0256],
         [-0.2217, -0.0156,  0.4536, -0.4472],
         [-0.0838, -0.4924,  0.0745,  0.0247],
         [ 0.2375, -0.4780,  0.3855, -0.2136],
         [-0.3364, -0.2123, -0.1836,  0.4425],
         [ 0.3200,  0.4839,  0.1815, -0.4457],
         [ 0.4992, -0.0481, -0.0751, -0.4213]], requires_grad=True),
 Parameter containing:
 tensor([ 0.2045,  0.4381,  0.3455,  0.0183, -0.0587,  0.2297, -0.1840,  0.3434,
         -0.3068,  0.2191], requires_grad=True),
 Parameter containing:
 tensor([[-0.0918,  0.1891, -0.0334,  0.1853, -0.1538,  0.2245, -0.1660,  0.0854,
          -0.2037,  0.0687],
         [-0.1507, -0.2738, -0.2944, -0.2543,  0.2120, -0.2743, -0.2186, -0.0053,
           0.0276, -0.0507],
         [ 0.0887, -0.0363,  0.0649,  0.0668, -0.0282, -0.0424, -0.1369,  0.1828,
     


## Conclusão e Próximos Passos

Neste notebook, abordamos os conceitos básicos do PyTorch e do deep learning e aplicamos esse conhecimento na construção 
e treinamento de uma rede neural MLP para classificar o conjunto de dados Iris. O PyTorch oferece uma ampla gama de 
funcionalidades que facilitam a construção e o treinamento de modelos de deep learning, mas isso é apenas a ponta do iceberg. 
Há muitos outros conceitos e técnicas em deep learning e PyTorch para explorar, incluindo diferentes tipos de redes neurais 
(como CNNs para processamento de imagem e RNNs para sequências de tempo), técnicas de regularização como dropout, e métodos 
de otimização avançados. Esperamos que este notebook sirva como um ponto de partida sólido para sua jornada no deep learning!
